# Nanopore base calling with Dorado

Jacobo de la Cuesta-Zuluaga. June 2025.

The aim of this notebook is to perform base calling from raw ONT `pod5` files.

## Before we start

The execution of the notebooks of this repo requires `conda` to be installed and an environment with `nextflow` available. You can find instructions about how to install conda [here](https://conda.io/projects/conda/en/latest/user-guide/install/index.html). 

## Load libraries and set paths

First, we'll set up the libraries and the work directory where we'll save our files


In [ ]:
# Libraries
library(tidyverse)
library(conflicted)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
conflicts_prefer(dplyr::filter())

[conflicted] Will prefer dplyr::filter over any other package.


The following chunk will define the directories where the data is stored and where the output will be saved. The present example assumes everything will be contained in the same directory: `base_dir`. This might be different in your particular case, for example, if your sequences are stored on a centralized directory or you have multiple runs stored in different folders. You can change this accordingly. 

In [ ]:
# Directories
# Base directory
base_dir = "/mnt/lustre/groups/maier/maide581/projects/Huequito"

# Data
data_dir = file.path(base_dir, "data")
dir.create(data_dir)

# Sequences
seq_dir = file.path(data_dir, "pod5_files")
dir.create(seq_dir)

# Out
out_dir = file.path(data_dir, "fastq_files")
dir.create(out_dir)

# software dir
bin_dir = file.path(base_dir, "bin")
dir.create(bin_dir)

# sheets dir
sheets_dir = file.path(data_dir, "sheets")
dir.create(sheets_dir)

# Software
conda_env = "nextflow"

Warning message in dir.create(data_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Huequito/data' already exists”
Warning message in dir.create(seq_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Huequito/data/pod5_files' already exists”
Warning message in dir.create(out_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Huequito/data/fastq_files' already exists”
Warning message in dir.create(sheets_dir):
“'/mnt/lustre/groups/maier/maide581/projects/Huequito/data/sheets' already exists”


## Download test files

For the present example, we'll use publicly available sequencing data from [Hall et al., 2024](https://doi.org/10.7554/eLife.98300.3). Specifically, this corresponds to the sequencing of an isolate of _Streptococcus dysgalactiae_. We'll use this sample to illustrate how to obtain `fastq` files from the `pod5` files of a demultiplexed sample.

This can take about half an hour

In [ ]:
Example_pod5 = "https://figshare.unimelb.edu.au/ndownloader/files/45334570"
Download_pod5_cmd = str_glue("wget --content-disposition --directory-prefix {seq_dir} {Example_pod5}", 
    seq_dir = seq_dir,
    Example_pod5 = Example_pod5)
#system(Download_pod5_cmd)

In [ ]:
# untar the file
tar_file = seq_dir %>% 
    list.files(full.names = TRUE,pattern = "tar")
untar_cmd = str_glue("tar -xf {tar_file} -C {seq_dir}",
    tar_file = tar_file,
    seq_dir = seq_dir)
#system(untar_cmd)

In [ ]:
# Final folder with actual pod5 files
raw_pod5_dir = file.path(seq_dir, "MMC234__202311")

## Download base calling software

Next, we need to download `dorado`, which is the software we'll use to perform the base calling.

Make sure you're using the latest version [here](https://github.com/nanoporetech/dorado)

In [ ]:
# Dorado file
dorado_url = "https://cdn.oxfordnanoportal.com/software/analysis/dorado-1.0.2-linux-x64.tar.gz"
dorado_destfile = file.path(bin_dir, basename(dorado_url))

# Download
download.file(url = dorado_url, destfile = dorado_destfile, method = "wget")

tar -zxf /mnt/lustre/groups/maier/maide581/projects/Huequito/bin/dorado-1.0.2-linux-x64.tar.gz -C /mnt/lustre/groups/maier/maide581/projects/Huequito/bin

In [ ]:
# Uncompress file
ungz_cmd = str_glue("tar -zxf {dorado_destfile} -C {bin_dir}",
    dorado_destfile = dorado_destfile,
    bin_dir = bin_dir)
#system(ungz_cmd)

In [ ]:
# Path to dorado executable
dorado_exec = file.path(str_remove(dorado_destfile, ".tar.gz"), "bin/dorado")

## Perform the base calling

For instructions on how to use `dorado` [see here](https://dorado-docs.readthedocs.io).

In the following chunks, we'll generate the slurm scripts necessary to execute dorado using GPUs on the M3 cluster. We'll need to complete some fields and specify certain file names or parameters.

In [ ]:
# Specify the name of the output file
# Since this will be compressed, be sure to include the `.fastq.gz` extension
fastq_filename = "MMC234_202311.fastq.gz"

In [ ]:
# Template slurm file
# Do not modify this chunk
dorado_slurm_raw = str_glue(.open = "[", .close = "]",
"#!/bin/bash
##############################
#       Parameters           #
##############################

# This section will tell the cluster what are the resources your job will need.
# Change the parameters accordingly and carefully!

# The success of your job depends on what you specify here.
# If you don't allocate enough resources (e.g. memory, cpus) your job will fail.
# If you allocate too much when not needed, your job will have a lower priority.

# The parameters here are a sensible start.

# Name of the job
#SBATCH --job-name=[[job_name]]

# Generate an output file and give it a name
#SBATCH --output=%x-%j.out

# Number of tasks
#SBATCH --ntasks=1

# Number of cpus that this task will need
#SBATCH --cpus-per-task=[[cpu]]

# Specify the total memory required per node
#SBATCH --mem=[[memory]]

# Specify the maximum time this job can take to run before being killed (hh:mm:ss)
#SBATCH --time=23:59:00

# Specify the partition to use
#SBATCH --partition=gpu-a30

# Type and number of gpus
#SBATCH --gres=gpu:2           

# job information
scontrol show job ${SLURM_JOB_ID}
pwd

# per node
# do your real computation
source $HOME/.bashrc
cd [[fastq_dir]]
[[dorado_exec]] basecaller [[accuracy_mode]] [[pod5_dir]] --emit-fastq --trim [[trim_option]] --device cuda:all | gzip > [[out_fastq_gz]]
")

In [ ]:
dorado_slurm = str_glue(dorado_slurm_raw, 
    job_name = "basecall_dorado",
    cpu = "16",
    memory = "128G",
    fastq_dir = out_dir,
    dorado_exec = dorado_exec,
    accuracy_mode = "hac",
    pod5_dir = raw_pod5_dir,
    trim_option = "all",
    out_fastq_gz = fastq_filename,
    .open = "[", .close = "]")

dorado_slurm %>%
    print()

#!/bin/bash
##############################
#       Parameters           #
##############################

# This section will tell the cluster what are the resources your job will need.
# Change the parameters accordingly and carefully!

# The success of your job depends on what you specify here.
# If you don't allocate enough resources (e.g. memory, cpus) your job will fail.
# If you allocate too much when not needed, your job will have a lower priority.

# The parameters here are a sensible start.

# Name of the job
#SBATCH --job-name=basecall_dorado

# Generate an output file and give it a name
#SBATCH --output=%x-%j.out

# Number of tasks
#SBATCH --ntasks=1

# Number of cpus that this task will need
#SBATCH --cpus-per-task=16

# Specify the total memory required per node
#SBATCH --mem=128G

# Specify the maximum time this job can take to run before being killed (hh:mm:ss)
#SBATCH --time=23:59:00

# Specify the partition to use
#SBATCH --partition=gpu-a30

# Type and number of gpus


In [ ]:
# Write slurm file
dorado_slurmfile = file.path(sheets_dir, "dorado_slurm.sh")
write_lines(dorado_slurm, dorado_slurmfile)

In [ ]:
# Execution command
str_glue("sbatch {dorado_slurmfile}")

sbatch /mnt/lustre/groups/maier/maide581/projects/Huequito/data/sheets/dorado_slurm.sh